# Getting started with Anonymouus : Example with test data #

To run the code below:
    
    1. Click on the cell to select it
    2. Press SHIFT+ENTER on your keyboard or press the play button in the toolbar above

In [2]:
import csv
from pathlib import Path
import pandas as pd
import re

from anonymouus import Anonymize

### Provide the data to be anonymized ###

In [5]:
test_data = Path('../tests/test_data/')

### Provide the mapping ###

In the mapping you specify the keywords to be replaced and their substitutes.
The mapping can be a: 
1. a dictionary 
2. the path to a csv file 
3. a function 

When using a csv-file, make sure: 
* your file has two columns 
    * left column: words to be replaced, 
    * right column: substitutions. 
* your file has a column header (any format)

For example:

In [7]:
key_csv = test_data/'keys.csv'

In [30]:
df_key = pd.read_csv(key_csv,index_col='names')
df_key

,subt
names,
Jane Doe,aaaa
Amsterdam,bbbb
j.doe@gmail.com,cccc
r#ca.*?er,dddd


Note that besides strings, you can also add regular expressions as keywords
* In a csv file: ```r#_my regex_```
* In a dictionary:```re.compile('_my regex_')``` 

In [19]:
key_dict = {
'Jane Doe': 'aaaa',
'Amsterdam':'bbbb',
'j.doe@gmail.com':'cccc',
re.compile('ca.*?er'):'dddd'
} 

### Create Anonymize object ###

In [21]:
anym = Anonymize(key_csv)

Customize the replacement process by adding options or flags

* Replace only entire words ```Anonymize(key_csv,use_word_boundaries=True)```
* Replace case-insensitive ```Anonymize(key_csv,flags=re.IGNORECASE)```

### Anonymize data ###

The replacement is carried out by the subsititute function. 
Provide the path to the folder with
* testdata (source)
* resultdata (target); will be created if it doesn't exist

In [31]:
anym.substitute(test_data,Path.cwd()/'anonymized/')